In [3]:
import numpy as np
import pandas as pd
import openpyxl
import sklearn

In [4]:
def get_var_name(var):
    for name, value in globals().items():
        if value is var:
            return name
 


In [5]:
ust = pd.read_excel("Data/HKUST Comp.xlsx").dropna().loc[:, ["Code","Title","Description"]]
hku = pd.read_csv("Data/HKU Comp.csv").dropna()
cuhk = pd.read_excel("Data/CUHK Comp.xlsx").dropna()


In [6]:
unilist=[ust,hku,cuhk]
for uni in unilist:
    uni["University"]=get_var_name(uni)


In [7]:
enrolleduni= input("Please enter the university you are enrolled in: ")
match enrolleduni:
    case "HKUST":
        otheruni = pd.concat([hku,cuhk])
        myuni=ust
        print(otheruni["Code"])
    case "HKU":
        otheruni = pd.concat([ust,cuhk])
        myuni=hku
        print(otheruni["Code"])
    case "CUHK":
        otheruni = pd.concat([ust,hku])
        myuni=cuhk
        print(otheruni["Code"])
    case _:
        print("Sorry, we do not have data for the university you entered.")
        exit()

otheruni = otheruni.drop_duplicates(subset=["Code"]).reset_index(drop=True)
print(otheruni)

0                COMP1117
1                ENGG1330
2                ENGG1340
3                FITE1010
4                FITE2000
             ...         
79    ENGG2740 / ESTR2016
80    ENGG2760 / ESTR2018
81    ENGG2780 / ESTR2020
82               ENGG3802
83               ENGG3803
Name: Code, Length: 130, dtype: object
                    Code                                              Title  \
0               COMP1117                               Computer Programming   
1               ENGG1330                             Computer Programming I   
2               ENGG1340                            Computer Programming II   
3               FITE1010             Introduction to Financial Technologies   
4               FITE2000                 Foundations of FinTech Programming   
..                   ...                                                ...   
125  ENGG2740 / ESTR2016  Differential Equations for Engineers (offered ...   
126  ENGG2760 / ESTR2018  Probability for E

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

correct = incorrect = mappingNA = user_input = 0
asked_courses = {}
documents = [otheruni["Description"][i] for i in range(len(otheruni))]
on_off = np.zeros(len(otheruni))
while user_input != "stop":
  user_input = input("Enter course code")
  if user_input == "stop":
    print("Thank you for using the program")
    break
  elif user_input not in myuni["Code"].values:
    print("The course code you entered is not in the system")
    continue
  
  if user_input not in asked_courses:

    documents.insert(0, myuni.loc[myuni['Code'] == user_input, 'Description'].iloc[0])
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(documents)
    cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)
    best_sim_score = np.delete(cosine_sim[0], 0)

    asked_courses[user_input] = best_sim_score

  #Finding the index of the most similar course
  best_three = np.argsort(best_sim_score)[-3:]
  for index in best_three:
    print(otheruni.loc[index, ["Code","Title","University"] ])

    user_input_2 = input("Is the course matching correct?")
    if user_input_2 == "yes":
      ...
    elif user_input_2 == "no":
    
      on_off[index] = 1

      print("A",best_sim_score)
      if best_sim_score.shape[0]==2: #Checking if the concatenated array already exists
        best_sim_score = np.array([best_sim_score[0],on_off])  #Then only taking the first dimension
      else: 
        best_sim_score = np.array([best_sim_score,on_off]) #If not, then concatenating without a worry

      indices = np.where(best_sim_score[1] == 0)[0]
      best_sim_score[0, indices] *= 1.1
      print("A",best_sim_score)


  

Code                                          CSCI1530
Title         Computer Principles and Java Programming
University                                        cuhk
Name: 78, dtype: object
A [0.1549961  0.15626666 0.18243379 0.08315454 0.17782177 0.18243379
 0.010732   0.13857467 0.08034831 0.12440043 0.07679296 0.0141333
 0.02974974 0.22982201 0.1239181  0.05626721 0.09978463 0.18295968
 0.12448202 0.09474596 0.09513365 0.13783685 0.17782177 0.13186284
 0.19764897 0.11107327 0.03429227 0.06727639 0.0811761  0.11686644
 0.07408503 0.21755425 0.105037   0.29691791 0.15200582 0.06530559
 0.         0.12121533 0.06135369 0.15734436 0.08703033 0.06121921
 0.00717216 0.13760045 0.16797187 0.13383246 0.09749571 0.22307071
 0.10751207 0.09649762 0.08918565 0.08528622 0.19547278 0.09441314
 0.07793152 0.16884241 0.10906989 0.07601733 0.07601733 0.09568448
 0.09568448 0.09985885 0.09809837 0.12864634 0.11295808 0.1067066
 0.10391142 0.09025816 0.11912892 0.07892001 0.07892001 0.20631567
 0.1847